In [1]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

import torch
import torch.nn as nn
import numpy as np
import os

# clip_grad_norm_ is used for truncated backpropagation
from torch.nn.utils import clip_grad_norm_


In [2]:
######################################################################
##################   DEVICE CONFIGURATION ############################
######################################################################

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
######################################################################
############ DEFINING THE DICTIIONARY AND CORPUS CLASSES #############
######################################################################

class Dictionary(object):
    
    def __init__(self):
        # Two dictionaries maintained
        # For finding the index of a given word - word2idx
        self.word2idx = {}
        # For finding the word given an index - idx2word
        self.idx2word = {}
        self.idx = 0
    
    # Method for adding a new word into an instance of the Dictionary class.
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    # method will return the vocabulary size.
    def __len__(self):
        return(len(self.word2idx))
    
class Corpus(object):
    def __init__(self):
        self.Dictionary = Dictionary()
        
    def get_data(self,path, batch_size=20):
        # Add word to the dictionary
        with open(path,'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.Dictionary.add_word(word)
        
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.Dictionary.word2idx[word]
                    token += 1 
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*(batch_size)]
        return(ids.view(batch_size, -1))
    

In [4]:
######################################################################
##################  HYPTERPARAMETER DEFINITION #######################
######################################################################

batch_size = 20
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 2
num_samples = 1000
seq_length = 30
learning_rate = 0.002


In [5]:
######################################################################
##############  LOADING THE PENN TREEBANK DATASET ####################
######################################################################

path = 'data/train.txt'
corpus = Corpus()
idxs = corpus.get_data(path, batch_size)
vocab_size = len(corpus.Dictionary)
num_batches = idxs.size(1) // seq_length

In [6]:
######################################################################
##############  DEFINING RNN BASED LANGUAGE MODEL ####################
######################################################################

class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        # Converting the one-hot encodings into the embeddings.
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        # Defining the computational graph for the Network.
        self.LSTM = nn.LSTM(embed_size, hidden_size, num_layers, batch_first = True)
        # Fully connected layer from the hidden to the vocabulary size.
        self.out = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        
        # Taking word indices to embeddings
        x = self.embeddings(x)
        
        # Forward propagate LSTM
        out, (h,c) = self.LSTM(x, h)
        
        # Reshaping the output in desired shape (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decoding to the final vocabulary size to get the word
        out = self.out(out)
        
        return(out, (h,c))
    
Lang_Model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [7]:
######################################################################
#######################  LOSS AND OPTIMIZER ##########################
######################################################################

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Lang_Model.parameters(), lr = learning_rate)

In [8]:
######################################################################
#################### TRUNCATED BACKPROPAGATION #######################
######################################################################

def detach(states):
    return [state.detach() for state in states]

In [9]:
######################################################################
####################### TRAINING THE MODEL ###########################
######################################################################

for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device), torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, idxs.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = idxs[:, i:i+seq_length].to(device)
        targets = idxs[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward Pass 
        states = detach(states)
        outputs, states = Lang_Model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backpropagation 
        Lang_Model.zero_grad()
        loss.backward()
        clip_grad_norm_(Lang_Model.parameters(), 0.5)
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'.format(epoch+1,num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))
        

Epoch: [1/2], Step: [0/1549], Loss: 9.2109, Perplexity: 10005.33
Epoch: [1/2], Step: [100/1549], Loss: 5.9969, Perplexity: 402.18
Epoch: [1/2], Step: [200/1549], Loss: 5.9242, Perplexity: 373.96
Epoch: [1/2], Step: [300/1549], Loss: 5.7676, Perplexity: 319.76
Epoch: [1/2], Step: [400/1549], Loss: 5.6344, Perplexity: 279.89
Epoch: [1/2], Step: [500/1549], Loss: 5.1213, Perplexity: 167.56
Epoch: [1/2], Step: [600/1549], Loss: 5.1859, Perplexity: 178.73
Epoch: [1/2], Step: [700/1549], Loss: 5.3598, Perplexity: 212.68
Epoch: [1/2], Step: [800/1549], Loss: 5.1754, Perplexity: 176.87
Epoch: [1/2], Step: [900/1549], Loss: 5.0794, Perplexity: 160.67
Epoch: [1/2], Step: [1000/1549], Loss: 5.1306, Perplexity: 169.13
Epoch: [1/2], Step: [1100/1549], Loss: 5.3766, Perplexity: 216.28
Epoch: [1/2], Step: [1200/1549], Loss: 5.1789, Perplexity: 177.49
Epoch: [1/2], Step: [1300/1549], Loss: 5.0874, Perplexity: 161.96
Epoch: [1/2], Step: [1400/1549], Loss: 4.8382, Perplexity: 126.24
Epoch: [1/2], Step: 

In [15]:
######################################################################
####################### TEST SET VALIDATION ##########################
######################################################################

with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set inital hidden and cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device), torch.zeros(num_layers, 1, hidden_size).to(device))
        
        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples = 1).unsqueeze(1).to(device)
        
        for i in range(num_samples):
            # Forward Propagation
            output, state = Lang_Model(input, state)
            
            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples = 1).item()
            
            # Fill input with samples word id for the next time step 
            input.fill_(word_id)
            
            # File Write
            word = corpus.Dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)
            
            if (i+1)%100 == 0:
                print('Sampled [{}\{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))
            
            

Sampled [100\1000] words and save to sample.txt
Sampled [200\1000] words and save to sample.txt
Sampled [300\1000] words and save to sample.txt
Sampled [400\1000] words and save to sample.txt
Sampled [500\1000] words and save to sample.txt
Sampled [600\1000] words and save to sample.txt
Sampled [700\1000] words and save to sample.txt
Sampled [800\1000] words and save to sample.txt
Sampled [900\1000] words and save to sample.txt
Sampled [1000\1000] words and save to sample.txt


In [ ]:
# Save the model checkpoints
torch.save(model.state_dict(), 'model.ckpt')